In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
from scipy.stats import f_oneway


# evaluate label propagation on the semi-supervised learning dataset

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']
Covariance=['spherical', 'tied', 'diag', 'full']
Params=['kmeans', 'random']
skf = StratifiedKFold(n_splits=8) 
fold=0

algo = 0
for i in Covariance:
        for j in Params:
                test=False
                for train_index, test_index in skf.split(X, y):
                        fold=fold+1
                
                        X_train, X_test = X[train_index,:], X[test_index,:]
                        y_train, y_test = y[train_index], y[test_index]
                        
                        gmm = GaussianMixture(n_components=2,init_params=j,covariance_type=i)
                        gmm.fit(X_train,y_train)
                        y_pred=gmm.predict(X_test)
                        if precision_score(y_test,y_pred)< 0.05:
    
                                y_pred = np.array([0 if label == 1 else 1 for label in y_pred])
                        
                        F1=round(f1_score(y_test,y_pred),3)
                       
                       
                        if test==False :
                                data = np.array([[F1]])
                                test=True
                        else :
                                data = np.append(data,np.array([F1]).reshape(1,1),axis=0)
                if algo == 0  :
                        df=pd.DataFrame(data,columns=['[{},{}]'.format(i,j)])
                        
                else :
                        df1=pd.DataFrame(data,columns=['[{},{}]'.format(i,j)])
                        df=pd.concat([df, df1], axis = 1)
                algo = algo + 1
display(df)
                       


              
                        
             


,"[spherical,kmeans]","[spherical,random]","[tied,kmeans]","[tied,random]","[diag,kmeans]","[diag,random]","[full,kmeans]","[full,random]"
0,0.896,0.896,0.800,0.289,0.909,0.909,0.822,0.896
1,0.923,0.923,0.152,0.346,0.923,0.923,0.779,0.779
2,0.857,0.857,0.921,0.474,0.882,0.882,0.811,0.845
3,0.933,0.933,0.982,0.766,0.889,0.889,0.829,0.866
4,0.923,0.923,0.949,0.391,0.896,0.896,0.822,0.857
5,0.984,0.984,0.966,0.410,0.984,0.984,0.952,0.984
6,0.938,0.938,0.824,0.657,0.921,0.921,0.896,0.896
7,0.968,0.968,0.909,0.648,0.938,0.938,0.896,0.896


In [67]:
t,p = f_oneway(df['[spherical,kmeans]'],df['[spherical,random]'],df['[tied,kmeans]'],df['[tied,random]'],df['[diag,kmeans]'],df['[diag,random]'],df['[full,kmeans]'],df['[full,random]'])
print('t statistic: %.3f' % t)
print('p value: %.13f' % p)


t statistic: 11.490
p value: 0.0000000061121
